# Оценка качества Retrieval: Recall@K и Hit Rate@K

Этот ноутбук выполняет оценку качества поискового движка:
- Загружает запросы из `queries.json` (по 5 синтетических запросов на каждый пост)
- Выполняет поиск топ-K документов для каждого запроса
- Считает **Recall@K** (Hit Rate@K) - долю запросов, где релевантный документ попал в топ-K
- **Сохраняет полный отчёт в `report.txt`** со всеми логами и результатами

## Методология
- **Ground truth**: для каждого запроса известен релевантный пост (из `queries.json`)
- **Recall@K** = 1 если ground truth документ найден в топ-K, иначе 0 (усреднено по всем запросам)
- **Hit Rate@K** = процент запросов, где найден релевантный документ (аналогично Recall@K)

## Выходные файлы
- `report.txt` - полный текстовый отчёт со всеми логами и результатами
- `eval_results_recall@K.json` - детальные результаты в JSON формате

## 1. Импорты и настройки

In [ ]:
import asyncio
import json
import logging
import sys
from pathlib import Path    
from typing import Dict, List
from datetime import datetime

# Добавляем путь к проекту
project_root = Path("/srv/nlp1/eval_dir/T-bank_NLP_")
sys.path.insert(0, str(project_root))

# Настройка логирования в файл и консоль
report_path = project_root / "src/eval/report.txt"

# Класс для дублирования вывода в файл и консоль
class DualLogger:
    def __init__(self, filepath):
        self.terminal = sys.stdout
        self.log = open(filepath, "w", encoding="utf-8")
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        self.log.flush()
        
    def flush(self):
        self.terminal.flush()
        self.log.flush()

# Перенаправляем stdout в файл и консоль
sys.stdout = DualLogger(report_path)

# Настраиваем logging для записи в тот же файл
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(report_path, mode='a', encoding='utf-8'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

print("=" * 80)
print(f"🚀 ОЦЕНКА КАЧЕСТВА RETRIEVAL - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)
print(f"📁 Корневая директория проекта: {project_root}")
print(f"📝 Отчёт сохраняется в: {report_path}")

🚀 ОЦЕНКА КАЧЕСТВА RETRIEVAL - 2025-11-16 20:05:41
📁 Корневая директория проекта: /srv/nlp1/eval_dir/T-bank_NLP_
📝 Отчёт сохраняется в: /srv/nlp1/eval_dir/T-bank_NLP_/src/eval/report.txt


✅ Импорты выполнены успешно
🔄 Инициализация RetrieverService...
✅ RetrieverService инициализирован
📊 Загружено 4991 запросов
✅ Будет обработано 4500 запросов

Пример запроса: Запущено ли сообщество для инвесторов в приложении Альфа-Инвестиций только для Android или планируется и для iOS?
Ground truth: channel=1418181070, message=7078
✅ Функция evaluate_query определена
🚀 Начало оценки для 4500 запросов (top_k=5)

[100/4500] Промежуточный Recall@5: 0.5600 (56.00%)
2025-11-16 20:07:02,181 - ERROR - Ошибка при поиске: 

[200/4500] Промежуточный Recall@5: 0.5500 (55.00%)
2025-11-16 20:07:44,346 - ERROR - Ошибка при поиске: 

[300/4500] Промежуточный Recall@5: 0.5367 (53.67%)

[400/4500] Промежуточный Recall@5: 0.5675 (56.75%)

[500/4500] Промежуточный Recall@5: 0.5680 (56.80%)
2025-11-16 20:08:36,660 - ERROR - Ошибка при поиске: 

[600/4500] Промежуточный Recall@5: 0.5700 (57.00%)

[700/4500] Промежуточный Recall@5: 0.5714 (57.14%)

[800/4500] Промежуточный Recall@5: 0.5637 (56.38%)

[900/

In [2]:
from tplexity.retriever.retriever_service import RetrieverService
from tqdm import tqdm

print("✅ Импорты выполнены успешно")

/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Инициализация RetrieverService

In [3]:
# Инициализация RetrieverService
print("🔄 Инициализация RetrieverService...")
retriever = RetrieverService()
print("✅ RetrieverService инициализирован")

## 3. Загрузка запросов из queries.json

In [ ]:
# Загружаем queries
queries_path = project_root / "src/eval/eval_data/queries.json"

with open(queries_path, "r", encoding="utf-8") as f:
    all_queries = json.load(f)

print(f"📊 Загружено {len(all_queries)} запросов")

# Для тестирования можно взять подмножество
NUM_QUERIES = 4991  # Установите число для теста, или None для всех запросов
queries_subset = all_queries[:NUM_QUERIES] if NUM_QUERIES else all_queries

# Фильтруем запросы: берем только индексы 0,1,2,5,6,7,10,11,12,...
# Паттерн: берем 3 элемента, пропускаем 2 (условие: индекс % 5 < 3)
queries = [q for i, q in enumerate(queries_subset) if i % 5 < 3]

print(f"✅ После фильтрации (индексы 0,1,2,5,6,7,10,11,12...): {len(queries)} запросов")
print(f"📈 Это {len(queries)/len(queries_subset)*100:.1f}% от исходного набора")
print(f"\nПример запроса: {queries[0]['query']}")
print(f"Ground truth: channel={queries[0]['id_channel']}, message={queries[0]['id_message']}")

## 4. Функция оценки для одного запроса

In [5]:
async def evaluate_query(query_data: dict, retriever, top_k: int = 5) -> dict:
    """
    Оценивает качество поиска для одного запроса
    
    Args:
        query_data: Данные запроса из queries.json
        retriever: RetrieverService
        top_k: Количество документов для оценки
    
    Returns:
        dict: Результаты оценки (recall, found_ground_truth, позиция GT документа)
    """
    query_text = query_data["query"]
    ground_truth_id = f"{query_data['id_channel']}_{query_data['id_message']}"
    
    # Выполняем поиск
    try:
        search_results = await retriever.search(
            query=query_text,
            top_k=top_k,
            top_n=top_k,
            use_rerank=False
        )
    except Exception as e:
        logger.error(f"Ошибка при поиске: {e}")
        return {
            "query": query_text,
            "ground_truth_id": ground_truth_id,
            "recall": 0.0,
            "hit": 0,
            "found_ground_truth": False,
            "gt_position": -1,
            "error": str(e)
        }
    
    if not search_results:
        return {
            "query": query_text,
            "ground_truth_id": ground_truth_id,
            "recall": 0.0,
            "hit": 0,
            "found_ground_truth": False,
            "gt_position": -1
        }
    
    # Проверяем, есть ли ground truth документ в топ-K
    found_ground_truth = False
    gt_position = -1
    
    for pos, (doc_id, score, text, metadata) in enumerate(search_results, start=1):
        # Получаем doc_id из метаданных
        doc_id_from_metadata = metadata.get("doc_id", "") if metadata else ""
        
        if doc_id_from_metadata == ground_truth_id:
            found_ground_truth = True
            gt_position = pos
            break
    
    # Recall@K = 1 если нашли ground truth, иначе 0
    recall = 1.0 if found_ground_truth else 0.0
    hit = 1 if found_ground_truth else 0
    
    return {
        "query": query_text,
        "ground_truth_id": ground_truth_id,
        "recall": recall,
        "hit": hit,
        "found_ground_truth": found_ground_truth,
        "gt_position": gt_position,
        "top_k_ids": [metadata.get("doc_id", "") if metadata else "" for _, _, _, metadata in search_results]
    }

print("✅ Функция evaluate_query определена")

## 5. Запуск оценки для всех запросов

In [ ]:
# Конфигурация
TOP_K = 5

print(f"🚀 Начало оценки для {len(queries)} запросов (top_k={TOP_K})")
print("=" * 80)

# Списки для хранения результатов
all_results = []
all_recalls = []
all_hits = []

# Обрабатываем каждый запрос
for i, query_data in enumerate(tqdm(queries, desc="Оценка запросов")):
    result = await evaluate_query(query_data, retriever, top_k=TOP_K)
    
    all_results.append(result)
    all_recalls.append(result["recall"])
    all_hits.append(result["hit"])
    
    # Логируем каждый 100-й результат
    if (i + 1) % 100 == 0:
        current_recall = sum(all_recalls) / len(all_recalls)
        print(f"\n[{i+1}/{len(queries)}] Промежуточный Recall@{TOP_K}: {current_recall:.4f} ({current_recall*100:.2f}%)")

print("\n" + "=" * 80)
print("✅ Оценка завершена!")

Оценка запросов: 100%|██████████| 4500/4500 [12:42<00:00,  5.90it/s]  


## 6. Вычисление итоговых метрик

In [7]:
# Вычисляем метрики
recall_at_k = sum(all_recalls) / len(all_recalls) if all_recalls else 0.0
hit_rate_at_k = sum(all_hits) / len(all_hits) if all_hits else 0.0
total_hits = sum(all_hits)

# Выводим результаты
print("=" * 80)
print("📊 ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("=" * 80)
print(f"Количество запросов: {len(queries)}")
print(f"Top-K: {TOP_K}")
print(f"\n🎯 Метрики:")
print(f"  Recall@{TOP_K}:    {recall_at_k:.4f} ({recall_at_k*100:.2f}%)")
print(f"  Hit Rate@{TOP_K}:  {hit_rate_at_k:.4f} ({hit_rate_at_k*100:.2f}%)")
print(f"  Найдено GT документов: {total_hits}/{len(queries)}")
print("\n📝 Интерпретация:")
print(f"  - Recall@{TOP_K} показывает долю запросов, где ground truth документ попал в топ-{TOP_K}")
print(f"  - Hit Rate@{TOP_K} = Recall@{TOP_K} (так как у каждого запроса только 1 релевантный документ)")
print("=" * 80)

## 7. Детальный анализ результатов

In [8]:
# Статистика по найденным документам
found_count = sum(1 for r in all_results if r.get("found_ground_truth", False))
print(f"\n📈 Детальная статистика:")
print(f"  Ground truth документ найден: {found_count}/{len(queries)} ({found_count/len(queries)*100:.1f}%)")
print(f"  Ground truth документ НЕ найден: {len(queries)-found_count}/{len(queries)} ({(len(queries)-found_count)/len(queries)*100:.1f}%)")

# Распределение позиций GT документа
from collections import Counter
positions = [r.get("gt_position", -1) for r in all_results if r.get("found_ground_truth", False)]

if positions:
    distribution = Counter(positions)
    print(f"\n📊 Распределение позиций найденных GT документов в топ-{TOP_K}:")
    for pos in sorted(distribution.keys()):
        freq = distribution[pos]
        print(f"  Позиция {pos}: {freq} запросов ({freq/found_count*100:.1f}% от найденных)")
    
    avg_position = sum(positions) / len(positions)
    print(f"\n  Средняя позиция GT документа (когда найден): {avg_position:.2f}")

## 8. Примеры результатов

In [9]:
# Показываем примеры успешных и неудачных запросов
print("\n" + "=" * 80)
print("📋 ПРИМЕРЫ РЕЗУЛЬТАТОВ")
print("=" * 80)

# Успешные и неудачные примеры
successful = [r for r in all_results if r.get("found_ground_truth", False)]
failed = [r for r in all_results if not r.get("found_ground_truth", False)]

print(f"\n✅ Примеры УСПЕШНЫХ запросов (GT найден в топ-{TOP_K}):")
print("-" * 80)
for i, result in enumerate(successful[:3], 1):
    print(f"\n[Пример {i}]")
    print(f"Query: {result['query'][:80]}...")
    print(f"Ground truth ID: {result['ground_truth_id']}")
    print(f"Позиция GT: {result['gt_position']}")
    print(f"Найденные doc_ids: {result['top_k_ids']}")

print(f"\n\n❌ Примеры НЕУДАЧНЫХ запросов (GT НЕ найден в топ-{TOP_K}):")
print("-" * 80)
for i, result in enumerate(failed[:3], 1):
    print(f"\n[Пример {i}]")
    print(f"Query: {result['query'][:80]}...")
    print(f"Ground truth ID: {result['ground_truth_id']}")
    print(f"Найденные doc_ids: {result['top_k_ids']}")

## 9. Сохранение результатов

In [ ]:
# Сохраняем результаты в JSON
output_data = {
    "config": {
        "num_queries": len(queries),
        "top_k": TOP_K,
        "use_rerank": False,
        "timestamp": datetime.now().isoformat()
    },
    "metrics": {
        f"recall@{TOP_K}": recall_at_k,
        f"hit_rate@{TOP_K}": hit_rate_at_k,
        "total_hits": total_hits,
        "total_queries": len(queries)
    },
    "detailed_results": all_results
}

output_path = project_root / f"src/eval/eval_results_recall@{TOP_K}.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Результаты сохранены в: {output_path}")
print(f"\n📊 Итоговые метрики:")
print(f"  Recall@{TOP_K} = {recall_at_k:.4f} ({recall_at_k*100:.2f}%)")
print(f"  Hit Rate@{TOP_K} = {hit_rate_at_k:.4f} ({hit_rate_at_k*100:.2f}%)")

print("\n" + "=" * 80)
print(f"📝 ПОЛНЫЙ ОТЧЁТ СОХРАНЁН В: {report_path}")
print(f"📋 JSON РЕЗУЛЬТАТЫ: {output_path}")
print("=" * 80)

# Закрываем файл логирования
if hasattr(sys.stdout, 'log'):
    sys.stdout.log.close()
    print("✅ Отчёт успешно сохранён!")

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/ipykernel/utils.py", line 60, in run_in_context
    return await asyncio.create_task(coro, context=context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 614, in shell_main
    await self.dispatch_shell(msg, subshell_id=subshell_id)
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 484, in dispatch_shell
    sys.stdout.flush()
  File "/tmp/ipykernel_1121459/1879823494.py", line 29, in flush
    self.log.flush()
ValueError: I/O operation on closed file.
